<a href="https://colab.research.google.com/github/oaustegard/Lottie_Playlist/blob/main/%F0%9F%8C%88_Playlist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌈 Sort your Spotify Playlist By Color 🌈 
### As inspired by Lottie

<br>

#### Question: Can you sort a Spotify Playlist by the color of each track's album cover?

#### Answer: Yes, at least _sort-a_ 
<br>
<small>(Dad Joke intended)</small>

Let's start by installing and importing the necessary libraries.  If run for the first time will ask for permissions to access Google Drive. This is for accessing the Spotify credentials in a /content/grive/vars.env file, picked up by `colab_env`.

If running this in your own GDrive, see the [SpotiPy Authentication](https://spotipy.readthedocs.io/en/2.19.0/#authorization-code-flow) and [Colab Env](https://github.com/apolitical/colab-env) instructions for what to put in your own vars.env file

Not necessary for subsequent invocations.

In [ ]:
try:
    import colab_env # see https://github.com/apolitical/colab-env for more details 
except:
    !pip install colab-env -qU
    import colab_env

try:
    import spotipy
except:
    !pip install spotipy
    import spotipy

import pandas as pd
import os
import requests
import pprint
from PIL import Image
from IPython.display import display
import webbrowser
pp = pprint.PrettyPrinter(indent=2)

In [ ]:
#@title Spotify Details
#@markdown Enter your Spotify User Id (which you can find here: https://www.spotify.com/us/account/overview/)
user_id = "122543813" #@param {type:"string"}
#@markdown Then enter your source Playlist Id (which is the last segment of the url when selecting a playlist at https://open.spotify.com/)
playlist_id = "5dxn0i8MPl6XFVVxNatd6U" #@param {type:"string"}


# Auth
Set the scope and authenticate as a Spotify user, applying the secrets stored as environmnet variables

In [ ]:
scope = 'playlist-modify-private playlist-modify-public' # see https://developer.spotify.com/documentation/general/guides/authorization/scopes/
sp = spotipy.Spotify(auth_manager=spotipy.SpotifyOAuth(username=user_id, scope=scope, open_browser=False))

# Playlist
Get the source playlist and its tracks, iterating until we have all of them.
<br><br>

#### Note
The first time this runs it will complete the Spotify OAuth process and may ask you to permit access to modify your playlists

In [ ]:
fields = 'name,tracks.total'
source_playlist_id = f'spotify:user:spotifycharts:playlist:{playlist_id}'

# get the playlist; we only need the name and the number of tracks
pl_results = sp.playlist(source_playlist_id, fields='name,tracks.total')
playlist_name = pl_results['name']
playlist_length = pl_results['tracks']['total']

In [ ]:
# Get the tracks from the playlist
offset = 0
playlist_items = []
# loop through the playlist until we get all the tracks
while offset < playlist_length:
    batch = sp.playlist_tracks(source_playlist_id, offset=offset, 
                fields='items(track(id,track_number,album(images)))')
    batch_items = batch['items']
    offset += len(batch_items)
    playlist_items.extend(batch_items)
#end while

# Color Logic
Define a number of functions to be used in the extraction of colors from the album covers and the subsequent sortoing of these. For more on color sorting and how the methods below were derived, see the [🔴Color🟢Sorting🔵.ipynb](https://colab.research.google.com/drive/1s1hMtukMIRjDmGApmSHvUSr_rnbC2tA7?usp=sharing) notebook. 

In [ ]:
def append_row(df:pd.DataFrame, row:list)->None:
    '''Append a row to a dataframe
    :param df: dataframe to append to
    :param row: row to append'''
    df.loc[len(df.index)] = row

def normalize_color(rgb:tuple)->tuple:
    """
    Normalize [0,255] color to [0,1] space
    """
    # use list comprehension to convert [0,255] color to [0,1] space
    return tuple([x/255 for x in rgb])

def rgb_to_hsY(rgb:tuple)->tuple:
    """Convert an RGB color to Hue, Saturation, and Perceived Luminance
    Use the HSL algorithm from ColorSys and the Perceived Luminance
    algorithm from https://en.wikipedia.org/wiki/Relative_luminance 
    :param rgb: RGB color as a tuple of [0, 1]-space values
    :return: a tuple with the hue in [0, 360]-space, saturation 
    and perceived brightness in the same [0, 1]-space as the input color"""
    r, g, b = rgb
    maxc = max(r, g, b)
    minc = min(r, g, b)
    sumc = (maxc+minc)
    rangec = (maxc-minc)
    l = sumc/2.0
    if minc == maxc:
        return 0.0, l, 0.0
    if l <= 0.5:
        s = rangec / sumc
    else:
        s = rangec / (2.0-sumc)
    rc = (maxc-r) / rangec
    gc = (maxc-g) / rangec
    bc = (maxc-b) / rangec
    if r == maxc:
        h = bc-gc
    elif g == maxc:
        h = 2.0+rc-bc
    else:
        h = 4.0+gc-rc
    h = int((h/6.0) % 1.0 * 360.0) # I don't quite grok the %1.0 part, but it seems to work

    # See https://en.wikipedia.org/wiki/Relative_luminance 
    Y = 0.2126*r**2.2 + 0.7152*g**2.2 + 0.0722*b**2.2
    
    return h, s, Y
#end def

def is_vivid(s:float, Y:float)->bool:
    """Determine if a color is 'vivid' based on the saturation and perceived brightness.
    Vivid thresholds based on the super-scientific approach of one person empirically 
    eyeballing colors of various saturation and brightnesss on an uncalibrated monitor, 
    using his own created online tool at https://jsfiddle.net/austegard/g1yobd4h/
    TODO: 
    :param s: saturation
    :param Y: perceived luminance
    :return: True if the color is 'vivid'"""

    return s > 0.15 and Y > 0.18 and Y < 0.95 
#end def

def get_rainbow_band(hue:float, band_deg:int)->int:
    """
    Get the "rainbow" band for a hue by dividing the hue color wheel into band_deg-sized partitions.
    Since the last 30º of the hue appears to this developer as more red than violet, 
    shift the hue wheel by 30º so they appear with the other reds at the beginning of the wheel 
    for rainbow like color bands
    :param hue: hue in [0, 360]-space
    :param band_size: size of the rainbow band partition in degrees
    :return: the rainbow band index
    """
    # apply the 30º shift
    rb_hue = (hue + 30) % 360
    # return the band index
    return rb_hue // band_deg

def get_image_rainbow_bands_and_perceived_brightness(image:Image, band_deg:int):
    """
    Get the rainbow bands (aka hue partitions) as a list of relative saturation for vivid colors 
    as well as the perceived brightness for an image
    :param image: PIL Image object
    :return: a tuple with the hue partitions as a list of floats and perceived brightness as a float
    """
    # convert the image to RGB and read the pixels
    pixels = image.convert('RGB').getdata()
    #TODO: for large images probably do some sampling to avoid memory issues, for now count on the images being small enough
    
    band_cnt = 360 // band_deg
    all_bands = dict.fromkeys(range(band_cnt), 0) 
    vivid_bands = dict.fromkeys(range(band_cnt), 0) 
    perceived_luminance = 0.0
    vivid_pixels = 0
    # for each pixel, get the hsY color, increment the total perceived luminance...
    for pixel in pixels:
        rgb = normalize_color(pixel)
        h, s, Y = rgb_to_hsY(rgb)
        perceived_luminance += Y
        # also capture the band in the case there are NO vivid colors (e.g. b/w)
        ab = get_rainbow_band(h, band_deg)
        all_bands[ab] += 1

        # ...and check if the color is vivid, if so get the rainbow band...
        if is_vivid(s, Y):
            vb = get_rainbow_band(h, band_deg)
            # ... and increment the color's band histogram value
            vivid_bands[vb] += 1  
            vivid_pixels += 1
        #end if
    #end for
    
    # get the bands to use (presumably this will normally be the vivid bands)
    if sum(vivid_bands.values()) > 0:
        bands = vivid_bands
        band_pixels = vivid_pixels
    else:
        bands = all_bands
        band_pixels = len(pixels)
    
    # normalize by dividing each value by the respective number of pixels to get [0,1]-space 
    # values to allow accurate comparison of different-sized images
    bands = {k:v/band_pixels for (k,v) in bands.items()}
    perceived_luminance = perceived_luminance / len(pixels)
    vividity = vivid_pixels / len(pixels)
    
    return bands, perceived_luminance, vividity
#end def

#get the primary color band from a bands dictionary to use for the hue partition
def get_primary_band(bands:dict)->int:
    """
    Get the primary band from a bands dictionary
    :param bands: bands dictionary
    :return: the primary band
    """
    return max(bands, key=bands.get) # I THINK I grok this one
 

# Process 

For each track, get the album cover image, and from that extract the primary vivid color's hue band, as well as the image's overall perceptive brightness.  
Add each to a dataframe, then sort on the hue band, brightness and track number and return the new track order 

In [ ]:
# Get the album covers for the tracks, extract color info and sort the tracks
df = pd.DataFrame(columns=['track_id', 'band', 'Y', 'vividity', 'track_number', 'img_url'])
# loop through the tracks in the playlist and get the smallest album cover for each track
#TODO: make this parallel to speed up the process? Lambdas could be a fun exercise...
for item in playlist_items:
    track = item['track']
    track_id = track['id']
    track_number = track['track_number']

    # conveniently the album cover images are always sorted by size, so we can just get 
    # the last one's url
    cover_image_url = track['album']['images'][-1]['url']
    # load the it as a PIL image
    track_image = Image.open(requests.get(cover_image_url, stream=True).raw).resize((10,10))


    # get the bands and perceived brightness
    bands, Y, vividity = get_image_rainbow_bands_and_perceived_brightness(track_image, band_deg=30)
    primary_band = get_primary_band(bands)
    # add the track to the dataframe
    append_row(df, [track_id, primary_band, Y, vividity, track_number, cover_image_url])
#end for

# sort the dataframe by the hue band and perceived brightness and finally track number 
# for multiple tracks from the same album
df = df.sort_values(by=['band', 'Y', 'track_number'])

#extract the resorted track_ids
sorted_track_ids = df['track_id'].tolist()

# Create 🌈 Playlist
Create the new 🌈-sorted Playlist

In [ ]:
# create a Spotify Playlist
new_playlist_name = f'🌈  {playlist_name} 🌈 '
new_playlist_description = f'The {playlist_name} playlist, sorted like a 🌈'
playlist = sp.user_playlist_create(user=user_id, public=False, 
                name=new_playlist_name, description=new_playlist_description)
#get the playlist external url
playlist_external_url = playlist['external_urls']['spotify']


In [ ]:
#pp.pprint(playlist)

Add the tracks in their new 🌈  order

In [ ]:
# %% add the tracks to the playlist
offset = 0
while offset < playlist_length:
    # Add the next batch of track ids - conveniently Python doesn't mind you asking for more than is available
    batch = sp.user_playlist_add_tracks(user=user_id, playlist_id=playlist['id'], tracks=sorted_track_ids[offset:offset+100])
    offset += 100
#end while

In [ ]:
# %% Display a link to the new playlist
print(playlist_external_url)

https://open.spotify.com/playlist/0yFW3MJoElYwMdlfRGZtAF


In [ ]:
# Experiment
# new_playlist_name = 'Extract low vividity albums'
# new_playlist_description = 'fofoo'
# old_df = df.copy()
# df.loc[df['vividity']<=0.1, 'band'] = 99
# df = df.sort_values(by=['band', 'Y', 'track_number'])

# 🌈 Covers
Since the Spotify UX does not lend itself to highlighting the color-sorting, show a larger copy of the covers here:

In [ ]:
# %% Local copy of the playlist - album-deduplified
img_df = df.drop_duplicates(subset=['img_url'])

img_template = '<img title="band: {band}, lum: {Y}, vividityy:{vividity}" src="{img_url}" />'
# apply the template to each row, return the results as a series, then concatenate the series to a string (phew)
imgs = img_df.apply(img_template.format_map, axis=1, result_type='reduce').str.cat(sep='\n')

ht = \
f'''<html><head><title>{new_playlist_name}</title>
<style>
  body {{font-family:Arial; color:#fff; background-color:#000;text-align:center}}
  div {{margin:0 auto; width:384px; max-width:384px}} 
  img {{width: 64px; height:64px;}}
</style>
</head>
<body><h1>{new_playlist_name}</h1>
<div>
{imgs}
</div>
</body></html>
''' 

# output the HTML to a file
safe_name = ''.join(filter(str.isalnum, playlist_name))
file_name = f'{safe_name}_rainbow.html'
with open(file_name, 'w') as f:
    f.write(ht)
#end with

In [ ]:
import IPython
IPython.display.HTML(filename=file_name)

In [ ]:
#TODO: Debug the 0 Y value of the Phil Collins album
#TODO: Why is this low lightness? https://i.scdn.co/image/ab67616d0000485122874c7fad7dee046bd69594
#TODO: Convert steps to Prefect DAG
#TODO: Option to use user's Liked songs as source: https://spotipy.readthedocs.io/en/2.19.0/#spotipy.client.Spotify.current_user_saved_tracks
#TODO: parallelize the slow image download + process step
#TODO: better extraction of "perceived color" from totality of image
#TODO: add a separate (or two?) additional band(s) for albums with low saturation/vividness 